# Building an ANN

In [5]:
# importing the libraries

import numpy as np
import pandas as pd
import tensorflow as tf

In [6]:
tf.__version__

'2.15.0'

### Data preprocessing

In [9]:
# importing the dataset

dataset = pd.read_csv("Churn_Modelling.csv")
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [10]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [11]:
print(y)

[1 0 1 ... 1 1 0]


In [12]:
# label encoding the "Gender" column

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [13]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [14]:
# One Hot Encoding the "Geography" column

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [15]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [18]:
# spliting the data into the Training set and Test set

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [20]:
# Feature Scaling

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Building the ANN

In [24]:
# initializing the ANN

ann = tf.keras.models.Sequential()

In [25]:
# adding the input layer and the first hidden layer

ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [26]:
# adding the second hidden layer

ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [27]:
# adding the output layer

ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

### Training the ANN

In [30]:
# Compiling the ANN

ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [31]:
# Training the ANN on the Training set

ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 0s 450us/step - loss: 0.6623 - accuracy: 0.5699
Epoch 2/100
250/250 [==============================] - 0s 304us/step - loss: 0.4973 - accuracy: 0.7935
Epoch 3/100
250/250 [==============================] - 0s 296us/step - loss: 0.4558 - accuracy: 0.7975
Epoch 4/100
250/250 [==============================] - 0s 297us/step - loss: 0.4352 - accuracy: 0.8008
Epoch 5/100
250/250 [==============================] - 0s 300us/step - loss: 0.4215 - accuracy: 0.8043
Epoch 6/100
250/250 [==============================] - 0s 295us/step - loss: 0.4078 - accuracy: 0.8110
Epoch 7/100
250/250 [==============================] - 0s 297us/step - loss: 0.3923 - accuracy: 0.8305
Epoch 8/100
250/250 [==============================] - 0s 296us/step - loss: 0.3791 - accuracy: 0.8386
Epoch 9/100
250/250 [==============================] - 0s 296us/step - loss: 0.3705 - accuracy: 0.8447
Epoch 10/100
250/250 [==============================] - 0s 443us/step - l

### Making the predictions and evaluating the model

In [33]:
# Predicting the result of a single observation

print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 [==============================] - 0s 11ms/step
[[False]]


In [35]:
# Predicting the Test set results

y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), axis=1))

63/63 [==============================] - 0s 269us/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [36]:
# Making the Confusing Matrix

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1517   78]
 [ 200  205]]


0.861